In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from tensorflow.keras import datasets, layers, models, regularizers, initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.io import imshow
from skimage.transform import rotate
from skimage.filters.edges import convolve

from skimage.io import imshow, imread
from skimage.transform import resize
from skimage.color import rgb2gray

import dill as pickle

np.random.seed(33)

In [2]:
df_test = pd.read_csv('data/test_labels.csv')
df_train = pd.read_csv('data/train_labels.csv')

bw_loaded = np.load('data/bw_images.npz')
X_train= bw_loaded['a']
X_test = bw_loaded['b']

X_test = X_test.reshape(-1,80,60,1)
X_train = X_train.reshape(-1,80,60,1)

In [3]:
# from model_functions import pick_ylabels, multi_index_counts, test_counts_by_cat, train_counts_by_cat, category_codes, class_weights

### Predict Article Types for Topwear

In [26]:
df_train.subCategory.value_counts().index

Index(['Topwear', 'Shoes', 'Bags', 'Watches', 'Bottomwear', 'Innerwear',
       'Eyewear', 'Jewellery', 'Fragrance', 'Flip Flops', 'Sandal', 'Socks',
       'Lips', 'Saree', 'Loungewear and Nightwear', 'Nails', 'Headwear',
       'Ties', 'Makeup'],
      dtype='object')

In [4]:
def cnn_article(type):
    sub_df = df_train[df_train.subCategory==type]
    sub_idx = list(sub_df.index)
    X_train= X_train[sub_idx]

    sub_df = df_train[df_train.subCategory==type]
    sub_idx = list(sub_df.index)
    X_train= X_train[sub_idx]


In [5]:
y_train0 = app_train.articleType.copy().astype('category').cat.codes
y_test0 = app_test.articleType.copy().astype('category').cat.codes

In [6]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train0)
y_test = to_categorical(y_test0)

In [7]:
print(y_train.shape)
print(y_test.shape)

(10394, 33)
(2603, 33)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [9]:
cnn_sub = models.Sequential()

In [10]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_sub.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_sub.add(layers.MaxPooling2D((2, 2)))
cnn_sub.add(Dropout(0.25))

# cnn_sub.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
# cnn_sub.add(MaxPooling2D(pool_size=(2, 2)))
# cnn_sub.add(Dropout(0.25))

cnn_sub.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_sub.add(MaxPooling2D(pool_size=(2, 2)))
cnn_sub.add(Dropout(0.25))

cnn_sub.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# cnn_sub.add(MaxPooling2D(pool_size=(2, 2)))
cnn_sub.add(Dropout(0.4))

cnn_sub.add(Flatten())

# cnn_sub.add(Dense(512, activation='relu'))
# cnn_sub.add(Dropout(0.5))
cnn_sub.add(Dense(128, activation='relu'))
cnn_sub.add(Dropout(0.3))
cnn_sub.add(Dense(33, activation='softmax'))

W0730 20:52:25.504179 140371908831040 deprecation.py:506] From /home/ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
cnn_sub.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 29, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 11, 128)       7

In [12]:
cnn_sub.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
cnn_sub.fit(X_train_app, y_train, epochs=10, validation_data=(X_test_app, y_test), verbose=1)

Train on 10394 samples, validate on 2603 samples
Epoch 1/10
10394/10394 [==============================] - 18s 2ms/sample - loss: 3.9848 - acc: 0.3913 - val_loss: 1.5069 - val_acc: 0.5828
Epoch 2/10
10394/10394 [==============================] - 14s 1ms/sample - loss: 1.4397 - acc: 0.6051 - val_loss: 1.0915 - val_acc: 0.6881
Epoch 3/10
10394/10394 [==============================] - 14s 1ms/sample - loss: 1.1432 - acc: 0.6771 - val_loss: 0.8817 - val_acc: 0.7391
Epoch 4/10
10394/10394 [==============================] - 14s 1ms/sample - loss: 0.9838 - acc: 0.7151 - val_loss: 0.7948 - val_acc: 0.7610
Epoch 5/10
10394/10394 [==============================] - 14s 1ms/sample - loss: 0.8629 - acc: 0.7433 - val_loss: 0.7186 - val_acc: 0.7733
Epoch 6/10
10394/10394 [==============================] - 14s 1ms/sample - loss: 0.7963 - acc: 0.7600 - val_loss: 0.6614 - val_acc: 0.7949
Epoch 7/10
10394/10394 [==============================] - 14s 1ms/sample - loss: 0.7407 - acc: 0.7805 - val_loss: 0.6

In [91]:
'''# save the fitted model
model_json = cnn_sub.to_json()
with open("cnn_apparel.json", "w") as json_file:
    json_file.write(model_json)'''

In [14]:
y_true = y_test0

In [15]:
# get the predictions for the test data
predicted_classes = cnn_sub.predict_classes(X_test_app)

# get the indices to be plotted
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


In [16]:
predicted_classes

array([11, 30, 21, ..., 11, 31, 11])

In [18]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(33)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       1.00      0.75      0.86         8
     Class 1       0.97      0.96      0.96        94
     Class 2       0.97      0.91      0.94       159
     Class 3       0.00      0.00      0.00         4
     Class 4       0.60      0.60      0.60        20
     Class 5       0.33      0.50      0.40         4
     Class 6       0.88      0.68      0.77        22
     Class 7       0.89      0.78      0.83        41
     Class 8       0.57      0.35      0.44        48
     Class 9       0.72      0.81      0.76       109
    Class 10       0.00      0.00      0.00         3
    Class 11       0.76      0.95      0.84       360
    Class 12       0.39      0.16      0.23        44
    Class 13       0.83      0.66      0.73        29
    Class 14       0.50      0.20      0.29        10
    Class 15       0.00      0.00      0.00         2
    Class 16       0.74      0.88      0.80        32
    Class 17       0.90    

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=7, width_shift_range=0.05, shear_range=0.05,
                               height_shift_range=0.05, zoom_range=0.1)
datagen = gen.flow(X_train_app, y_train)

In [22]:
test1 = cnn_sub.fit_generator(datagen, epochs = 50, use_multiprocessing=True)

Epoch 1/50
325/325 [==============================] - 22s 67ms/step - loss: 1.1353 - acc: 0.6689
Epoch 2/50
325/325 [==============================] - 20s 61ms/step - loss: 1.0064 - acc: 0.6987
Epoch 3/50
325/325 [==============================] - 20s 63ms/step - loss: 0.9309 - acc: 0.7218
Epoch 4/50
325/325 [==============================] - 21s 65ms/step - loss: 0.9062 - acc: 0.7229
Epoch 5/50
325/325 [==============================] - 21s 65ms/step - loss: 0.8680 - acc: 0.7341
Epoch 6/50
325/325 [==============================] - 21s 66ms/step - loss: 0.8327 - acc: 0.7472
Epoch 7/50
325/325 [==============================] - 22s 67ms/step - loss: 0.8109 - acc: 0.7537
Epoch 8/50
325/325 [==============================] - 21s 65ms/step - loss: 0.8293 - acc: 0.7459
Epoch 9/50
325/325 [==============================] - 21s 64ms/step - loss: 0.7723 - acc: 0.7619
Epoch 10/50
325/325 [==============================] - 20s 62ms/step - loss: 0.7611 - acc: 0.7563
Epoch 11/50
325/325 [========

### Predict Sub-Categories for Accessories

In [89]:
acc_train = df_train[df_train.masterCategory=='Accessories']
acc_train_idx = list(acc_train.index)
X_train_acc = X_train[acc_train_idx]

acc_test = df_test[df_test.masterCategory=='Accessories']
acc_test_idx = list(acc_test.index)
X_test_acc = X_test[acc_test_idx]

y_train_acc0 = acc_train.subCategory.copy().astype('category').cat.codes
y_test_acc0 = acc_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_acc = to_categorical(y_train_acc0)
y_test_acc = to_categorical(y_test_acc0)

print(y_train_acc.shape)
print(y_test_acc.shape)

(5586, 6)
(1656, 6)


In [84]:
cnn_acc = models.Sequential()

In [85]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_acc.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_acc.add(layers.MaxPooling2D((2, 2)))
cnn_acc.add(Dropout(0.2))

cnn_acc.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_acc.add(MaxPooling2D(pool_size=(2, 2)))
cnn_acc.add(Dropout(0.25))

cnn_acc.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_acc.add(Dropout(0.4))


cnn_acc.add(Flatten())

cnn_acc.add(Dense(128, activation='relu'))
cnn_acc.add(Dropout(0.3))
cnn_acc.add(Dense(6, activation='softmax'))

In [86]:
cnn_acc.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 11, 128)      

In [87]:
cnn_acc.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [90]:
cnn_acc.fit(X_train_acc, y_train_acc, epochs=10, validation_data=(X_test_acc, y_test_acc), verbose=1)

Train on 5586 samples, validate on 1656 samples
Epoch 1/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 5.3784 - acc: 0.7053 - val_loss: 0.2950 - val_acc: 0.8877
Epoch 2/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.3736 - acc: 0.8813 - val_loss: 0.1463 - val_acc: 0.9589
Epoch 3/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.2637 - acc: 0.9239 - val_loss: 0.2154 - val_acc: 0.9185
Epoch 4/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.2196 - acc: 0.9338 - val_loss: 0.1077 - val_acc: 0.9716
Epoch 5/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1816 - acc: 0.9440 - val_loss: 0.1520 - val_acc: 0.9662
Epoch 6/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1455 - acc: 0.9552 - val_loss: 0.1314 - val_acc: 0.9668
Epoch 7/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1347 - acc: 0.9603 - val_loss: 0.0842 - val_acc: 0.9819


In [92]:
y_true_acc = y_test_acc0

In [97]:
# get the predictions for the test data
predicted_classes_acc = cnn_acc.predict_classes(X_test_acc)

# get the indices to be plotted
correct = (predicted_classes_acc ==y_true_acc).to_numpy().nonzero()[0]
incorrect = (predicted_classes_acc !=y_true_acc).to_numpy().nonzero()[0]

In [98]:
from sklearn.metrics import classification_report
target_names_acc = ["Class {}".format(i) for i in range(6)]
print(classification_report(y_true_acc, predicted_classes_acc, target_names=target_names_acc))

              precision    recall  f1-score   support

     Class 0       0.99      0.97      0.98       597
     Class 1       1.00      1.00      1.00       213
     Class 2       1.00      0.79      0.88        56
     Class 3       0.93      0.98      0.96       230
     Class 4       1.00      0.98      0.99        50
     Class 5       0.97      0.99      0.98       510

    accuracy                           0.98      1656
   macro avg       0.98      0.95      0.96      1656
weighted avg       0.98      0.98      0.98      1656



In [99]:
# save the fitted model
model_json = cnn_acc.to_json()
with open("cnn_accessories.json", "w") as json_file:
    json_file.write(model_json)

### Predict Sub-Categories for Footwear

In [100]:
fw_train = df_train[df_train.masterCategory=='Footwear']
fw_train_idx = list(fw_train.index)
X_train_fw = X_train[fw_train_idx]

fw_test = df_test[df_test.masterCategory=='Footwear']
fw_test_idx = list(fw_test.index)
X_test_fw = X_test[fw_test_idx]

y_train_fw0 = fw_train.subCategory.copy().astype('category').cat.codes
y_test_fw0 = fw_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_fw = to_categorical(y_train_fw0)
y_test_fw = to_categorical(y_test_fw0)

print(y_train_fw.shape)
print(y_test_fw.shape)

(6296, 3)
(1796, 3)


In [106]:
cnn_fw = models.Sequential()

In [107]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_fw.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_fw.add(layers.MaxPooling2D((2, 2)))
cnn_fw.add(Dropout(0.25))

cnn_fw.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
cnn_fw.add(MaxPooling2D(pool_size=(2, 2)))
cnn_fw.add(Dropout(0.25))

cnn_fw.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_fw.add(MaxPooling2D(pool_size=(2, 2)))
cnn_fw.add(Dropout(0.25))

cnn_fw.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_fw.add(Dropout(0.25))


cnn_fw.add(Flatten())
cnn_fw.add(Dense(512, activation='relu'))
cnn_fw.add(Dropout(0.5))
cnn_fw.add(Dense(128, activation='relu'))
cnn_fw.add(Dropout(0.5))
cnn_fw.add(Dense(3, activation='softmax'))

In [108]:
cnn_fw.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 11, 128)      

In [109]:
cnn_fw.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_fw.fit(X_train_fw, y_train_fw, epochs=20, validation_data=(X_test_fw, y_test_fw), verbose=1)

Train on 6296 samples, validate on 1796 samples
Epoch 1/20
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.2411 - acc: 0.9096 - val_loss: 0.2692 - val_acc: 0.9059
Epoch 2/20
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.2430 - acc: 0.9142 - val_loss: 0.2402 - val_acc: 0.9209
Epoch 3/20
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.2186 - acc: 0.9196 - val_loss: 0.2267 - val_acc: 0.9259
Epoch 4/20
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.2075 - acc: 0.9187 - val_loss: 0.2224 - val_acc: 0.9287
Epoch 5/20
1152/6296 [====>.........................] - ETA: 6s - loss: 0.1787 - acc: 0.9297

In [92]:
y_true_fw = y_test_fw0

In [97]:
# get the predictions for the test data
predicted_classes_fw = cnn_fw.predict_classes(X_test_fw)

# get the indices to be plotted
correct = (predicted_classes_fw ==y_true_fw).to_numpy().nonzero()[0]
incorrect = (predicted_classes_fw !=y_true_fw).to_numpy().nonzero()[0]

In [98]:
from sklearn.metrics import classification_report
target_names_fw = ["Class {}".format(i) for i in range(3)]
print(classification_report(y_true_fw, predicted_classes_fw, target_names=target_names_fw))

              precision    recall  f1-score   support

     Class 0       0.99      0.97      0.98       597
     Class 1       1.00      1.00      1.00       213
     Class 2       1.00      0.79      0.88        56
     Class 3       0.93      0.98      0.96       230
     Class 4       1.00      0.98      0.99        50
     Class 5       0.97      0.99      0.98       510

    accuracy                           0.98      1656
   macro avg       0.98      0.95      0.96      1656
weighted avg       0.98      0.98      0.98      1656



In [99]:
# save the fitted model
model_json = cnn_fw.to_json()
with open("cnn_footwear.json", "w") as json_file:
    json_file.write(model_json)

### Predict Sub-Categories for Personal Care

In [100]:
pc_train = df_train[df_train.masterCategory=='Personal Care']
pc_train_idx = list(pc_train.index)
X_train_pc = X_train[pc_train_idx]

pc_test = df_test[df_test.masterCategory=='Personal Care']
pc_test_idx = list(pc_test.index)
X_test_pc = X_test[pc_test_idx]

y_train_pc0 = fw_train.subCategory.copy().astype('category').cat.codes
y_test_pc0 = fw_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_pc = to_categorical(y_train_pc0)
y_test_pc = to_categorical(y_test_pc0)

print(y_train_pc.shape)
print(y_test_pc.shape)

(6296, 3)
(1796, 3)


In [106]:
cnn_pc = models.Sequential()

In [107]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_pc.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_pc.add(layers.MaxPooling2D((2, 2)))
cnn_pc.add(Dropout(0.2)

cnn_pc.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_pc.add(MaxPooling2D(pool_size=(2, 2)))
cnn_pc.add(Dropout(0.25)

cnn_pc.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_pc.add(Dropout(0.4)
           
cnn_pc.add(Flatten()
           
cnn_pc.add(Dense(128, activation='relu'))
cnn_pc.add(Dropout(0.3))
cnn_pc.add(Dense(3, activation='softmax'))

In [108]:
cnn_fw.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 11, 128)      

In [109]:
cnn_fw.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_fw.fit(X_train_fw, y_train_fw, epochs=10, validation_data=(X_test_fw, y_test_fw), verbose=1)

Train on 6296 samples, validate on 1796 samples
Epoch 1/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 5.0477 - acc: 0.7851 - val_loss: 0.7035 - val_acc: 0.7973
Epoch 2/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.5417 - acc: 0.8135 - val_loss: 0.3989 - val_acc: 0.8190
Epoch 3/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.4569 - acc: 0.8313 - val_loss: 0.3964 - val_acc: 0.8330
Epoch 4/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.3950 - acc: 0.8523 - val_loss: 0.3420 - val_acc: 0.8803
Epoch 5/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.3457 - acc: 0.8774 - val_loss: 0.3733 - val_acc: 0.8469
Epoch 6/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.3220 - acc: 0.8809 - val_loss: 0.3126 - val_acc: 0.8959
Epoch 7/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.2981 - acc: 0.8939 - val_loss: 0.2801 - val_acc: 0.9087


In [92]:
y_true_fw = y_test_fw0

In [97]:
# get the predictions for the test data
predicted_classes_fw = cnn_fw.predict_classes(X_test_fw)

# get the indices to be plotted
correct = (predicted_classes_fw ==y_true_fw).to_numpy().nonzero()[0]
incorrect = (predicted_classes_fw !=y_true_fw).to_numpy().nonzero()[0]

In [98]:
from sklearn.metrics import classification_report
target_names_fw = ["Class {}".format(i) for i in range(3)]
print(classification_report(y_true_fw, predicted_classes_fw, target_names=target_names_fw))

              precision    recall  f1-score   support

     Class 0       0.99      0.97      0.98       597
     Class 1       1.00      1.00      1.00       213
     Class 2       1.00      0.79      0.88        56
     Class 3       0.93      0.98      0.96       230
     Class 4       1.00      0.98      0.99        50
     Class 5       0.97      0.99      0.98       510

    accuracy                           0.98      1656
   macro avg       0.98      0.95      0.96      1656
weighted avg       0.98      0.98      0.98      1656



In [99]:
# save the fitted model
model_json = cnn_fw.to_json()
with open("cnn_footwear.json", "w") as json_file:
    json_file.write(model_json)